# Lab 5 - Spark Streaming

Streaming is something that is rapidly advancing and changin fast, there are multipl enew libraries every year, new and different services always popping up, and what is in this notebook may or may not apply to you. Maybe your looking for something specific on Kafka, or maybe you are looking for streaming about twitter, in which case Spark might be overkill for what you really want. Realistically speaking each situation is going to require a customized solution and this course is never going to be able to supply a one size fits all solution. 

For those of you new to Spark Streaming, let's get started with a classic example, streaming Twitter! Twitter is a great source for streaming because its something most people already have an intuitive understanding of, you can visit the site yourself, and a lot of streaming technology has come out of Twitter as a company. You don't access to the entire "firehose" of twitter without paying for it, but that would be too much for us to handle anyway, so we'll be more than fine with the freely available API access.

-----

Let's discuss SparkStreaming!

Spark Streaming is an extension of the core Spark API that enables scalable, high-throughput, fault-tolerant stream processing of live data streams. Data can be ingested from many sources like Kafka, Flume, Kinesis, or TCP sockets, and can be processed using complex algorithms expressed with high-level functions like map, reduce, join and window. Finally, processed data can be pushed out to filesystems, databases, and live dashboards. In fact, you can apply Spark’s machine learning and graph processing algorithms on data streams.

<img src='http://spark.apache.org/docs/latest/img/streaming-arch.png'/>

There are SparkSQL modules for streaming: 

http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=streaming#module-pyspark.sql.streaming

But they are all still listed as experimental, so instead of showing you somethign that might break in the future, we'll stick to the RDD methods (which is what the documentation also currently shows for streaming).

Internally, it works as follows. Spark Streaming receives live input data streams and divides the data into batches, which are then processed by the Spark engine to generate the final stream of results in batches.

<img src='http://spark.apache.org/docs/latest/img/streaming-flow.png'/>

## Simple Local Example

We'll do a simple local counting example, make sure to watch the video for this, the example will only work on Linux type systems, not on a Windows computer. This makes sense because you won't run this on Windows in the real world. Definitely watch the video for this one, a lot of it can't be replicated on Jupyter Notebook by itself!

In [0]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext.getOrCreate();
ssc = StreamingContext(sc, 1)

In [0]:
# Create a DStream that will connect to hostname:port, like localhost:9999
# Firewalls might block this!
lines = ssc.socketTextStream("localhost", 9999)

In [0]:
# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

In [0]:
# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

Now we open up a Unix terminal and type:

         $ nc -lk 9999
     $ hello world any text you want
     
With this running run the line below, then type Ctrl+C to terminate it.

In [0]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

-------------------------------------------
Time: 2022-12-29 07:23:46
-------------------------------------------

-------------------------------------------
Time: 2022-12-29 07:23:47
-------------------------------------------
('hello', 1)
('text', 1)
('you', 1)
('world', 1)
('any', 1)
('want', 1)

-------------------------------------------
Time: 2022-12-29 07:23:48
-------------------------------------------

-------------------------------------------
Time: 2022-12-29 07:23:49
-------------------------------------------

-------------------------------------------
Time: 2022-12-29 07:23:50
-------------------------------------------

-------------------------------------------
Time: 2022-12-29 07:23:51
-------------------------------------------

-------------------------------------------
Time: 2022-12-29 07:23:52
-------------------------------------------

-------------------------------------------
Time: 2022-12-29 07:23:53
-------------------------------------------

--------

In [0]:
ssc.stop()

In [0]:
# ### Create SPARK_HOME and PYLIB env var and update PATH env var

import os
import sys

# ### Initializing Spark

# Build __SparkConf__ object 
# 
#     Contains information about your application.  
# 
# 
# Create __SparkContext__ object 
#     
#     Tells Spark how to access a cluster. 
#     
# 
# Create __SparkSession__ object
# 
#     The entry point to programming Spark with the Dataset and DataFrame API.
# 
#     Used to create DataFrame, register DataFrame as tables and execute SQL over tables etc.
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Import the necessary classes and create a local SparkSession, the starting point of all functionalities related to Spark.
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [0]:
spark=SparkSession.builder.appName('SparkStreaming_01').getOrCreate()

In [0]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()

In [0]:
# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

In [0]:
# Generate running word count
wordCounts = words.groupBy("word").count()
query = wordCounts.writeStream.outputMode("complete").format("memory").queryName("output").start()

In [0]:
from IPython.display import display, clear_output
from time import sleep

while True:
    clear_output(wait=True)
    display(query.status)
    display(spark.sql('SELECT * FROM output').show())
    sleep(10)

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}+-----------+-----+
|       word|count|
+-----------+-----+
|     launch|    2|
|    session|    2|
|   session.|    2|
|        new|    2|
|server-side|    2|
| Refreshing|    2|
|   terminal|    2|
|    timeout|    2|
|         is|    2|
|        the|    4|
|    client-|    2|
|      There|    2|
|        are|    2|
|       made|    2|
|         no|    2|
|        tab|    2|
|         an|    2|
|    changes|    2|
|      would|    2|
|         if|    2|
+-----------+-----+
only showing top 20 rows

None

In [0]:
query.awaitTermination()